In [ ]:
%run IMPORT.ipynb

In [ ]:
%config InlineBackend.figure_format = 'retina'
%matplotlib notebook

### Clumping, COJO, PAINTOR, VEP

#### Prepare files for Clumping

In [ ]:
summary = pd.read_pickle('data/healthspan.pkl')

In [ ]:
cojodf = summary.reset_index()
cojodf['TEST'] = 'ADD'

In [ ]:
cojodf = summary.rename(columns={'P':'P_notcorrected'})
cojodf['TEST'] = 'ADD'

lambda_gc = 1.0532
split = np.array_split(summary['STAT']/np.sqrt(lambda_gc),n_jobs)
concat = pd.concat(multiproc_pbar(parallel_p, [split], [True]))

cojodf['P'] = concat

In [ ]:
cojodir = 'healthspan'

In [ ]:
pbar = ProgressBar()
for chrom in pbar(cojodf['Chromosome'].unique()):
    chrslice = cojodf[cojodf['Chromosome']==chrom]
    chrslice = chrslice.drop_duplicates(subset=['Marker'], keep=False)
    chrslice.to_csv(
        '%s/chr%d.assoc.linear.gz' % (cojodir,int(chrom)),
               sep='\t', index=False, compression='gzip',
              columns=['Chromosome','Marker','Position','eff_allele','TEST','PhenoCount','Slope','STAT','P'],
             header=['CHR','SNP','BP','A1','TEST','NMISS','BETA','STAT','P'])

#### 1000g for PAINTOR ref

In [ ]:
for c {1..22}; do
    wget ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/ALL.chr$c.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz;
done

In [ ]:
for c in {1..22}; do
    zgrep -v '^#' ref/ALL.chr$c.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz | cut -f 3 | sort | uniq -d > ref/chr$c.dups;
    echo $c;
    wc -l ref/chr$c.dups;
    plink --vcf ref/ALL.chr$c.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz --exclude ref/chr$c.dups --keep EUR.txt --make-bed --out ref/chr$c;
done

#### Clumping

In [ ]:
cojodir=/data/cojo/andersen_list_1000g_2
refdir=/data/1000g/ref
parallel --joblog clumping.log --eta plink --bfile {.} --keep-allele-order \
--clump $cojodir/{/.}.assoc.linear.gz --clump-p1 5e-8 --maf 0.002 \
--clump-p2 5e-8 --clump-r2 0.1 --clump-range /data/UKBB/sup/glist-hg19 \
--out $cojodir/{/.} ::: $refdir/chr*bed

#### Prepares files for PAINTOR

In [ ]:
chdir('/data/')

In [ ]:
refdir='/data/10KREFS/10kref_imp_v2'
paintordir = '/data/paintor/paintor_test'

In [ ]:
mkdir(paintordir)
annotation_files = glob('/data/soft/dist/Functional_Annotations/GeneElements_Gencode/*')
np.savetxt('/data/files/paintor_annotation_files', annotation_files, fmt='%s')

def parallel(f):
    name = basename(f).split('.')[0]

    df = pd.read_csv('%s/%s.assoc.linear.gz' % (cojodir,name), delim_whitespace=True, dtype=str,
                     engine='c', memory_map=True, usecols=['SNP','BP','STAT','CHR'], index_col='SNP')
    clumps = pd.read_csv(f, delim_whitespace=True, index_col='SNP', usecols=['SNP','SP2','CHR','BP'],
                         na_values=['NONE'])
    for locus, sp2 in clumps.iterrows():
    #         if chromosome==6 and (24e6<position<35e6):
    #             continue
        snplist = []
        if sp2.notnull().SP2:
            snplist += map(lambda x: x.split('(')[0], sp2.SP2.split(','))
        snplist += [locus]
        data = df.ix[snplist]
        if len(data)<2:
            print data
            continue
        data['index'] = data.index
        data['chr'] = 'chr'+data.CHR
        data.rename(columns={'BP':'pos', 'STAT':'ZSCORE', 'index':'rsid'}, inplace=True)
#         print data
        data.to_csv('%s/%s'%(paintordir,locus), sep=' ', index=False)
        data.to_csv('%s/%s.snplist'%(paintordir,locus), sep=' ', index=False, columns=['rsid'], header=False)
        ldcmd = ['plink','--bfile','%s/%s'%(refdir,name),'--keep-allele-order','--extract',
                 '%s/%s.snplist'%(paintordir,locus),'--r','square','spaces','--out','%s/%s'%(paintordir,locus)]
        print(' '.join(ldcmd))
        subprocess.check_call(ldcmd)

        ld = pd.read_csv('%s/%s.ld'%(paintordir,locus), sep=' ', na_values=['nan'], header=None)
        ld.fillna(0).to_csv('%s/%s.ld'%(paintordir,locus), sep=' ', header=False, index=False)
        pd.DataFrame(np.ones(len(data)).astype(int), columns=['empty']).to_csv(
            '%s/%s.annotations'%(paintordir,locus), index=False)

flst = glob('%s/*.clumped'%cojodir)
for f in flst:
    print f
    parallel(f)
# multiproc_pbar(parallel, [flst], [True]);

In [ ]:
input_file = [f.split('.')[0].split('/')[-1] for f in glob('%s/*.ld'%paintordir)]
np.savetxt('%s/paintor.filelist'%paintordir, input_file, fmt='%s')

#### Runs PAINTOR

In [ ]:
annotations = [basename(elem) for elem in np.loadtxt('/data/files/paintor_annotation_files',
                                                     dtype=str)]

def parallel(annotation):
    ancmd = ['PAINTOR','-input','%s/paintor.filelist'%paintordir,'-Zhead','ZSCORE','-LDname','ld',
             '-in','%s/'%paintordir,'-out','%s/'%paintordir,
#              '-enumerate','2',
             '-RESname','%s_results'%annotation,
             '-Gname','%s/Enrich.%s'%(paintordir,annotation),
             '-Lname','%s/BF.%s'%(paintordir,annotation)]
    print ' '.join(ancmd)
    subprocess.check_call(ancmd)

parallel('empty')
# multiproc_pbar(parallel, [annotations], [True]);

In [ ]:
ids = pd.read_pickle('/data/files/500k_maf001_info09_snpdata.pkl').set_index('newindex')

In [ ]:
flst = glob('%s/*.empty_results'%paintordir)
mass = []
pbar = ProgressBar()

for f in pbar(flst):

    inp = pd.read_csv(f, sep=' ')
#     inp['rsid'] = inpt.loc[inp.rsid,'RSID'].values
    inp['chr'] = inp['chr'].apply(lambda x: x[3:])
    inp.drop(['ZSCORE'], axis=1, inplace=True)
    inp['locus_id'] = splitext(basename(f))[0]
    threshold = 0.99*inp.Posterior_Prob.sum()
    inp.sort_values('Posterior_Prob', ascending=False, inplace=True)
#     print inp.values[0,2]
    cumsum = inp.Posterior_Prob.cumsum()
    for i, elem in enumerate(cumsum.values):
        if cumsum.values[i]<threshold and cumsum.values[i+1] >threshold:
            new_threshold = cumsum.values[i+1]
            break
    ix = cumsum.index[:i+2]
    inp['above_threshold'] = False
    inp.loc[ix,'above_threshold'] = True
    mass.append(inp)

df = pd.concat(mass, ignore_index=False).reset_index().drop(['index'], axis=1)

In [ ]:
ids = pd.concat({chrom: pd.read_csv('/data/UKBB/imputed/ukb_mfi_chr%s_v3.txt'%chrom,
                   sep='\t', header=None,
                 usecols=(1,2,3,4)) for chrom in range(1,23)}).reset_index().drop(['level_1'], axis=1)

ids['newindex'] = ids['level_0'].astype(str)+':'+ids[2].astype(str)+'_'+ids[3]+'_'+ids[4]
# bims = pd.concat([pd.read_csv(f, sep='\t', header=None) for f in glob('/data/10KREFS/10kref_imp_v2/chr*bim')])
ids[[1,'newindex']].set_index('newindex')[1].to_csv('/data/files/newindex2rsid.csv')

In [ ]:
ids = pd.read_csv('/data/files/newindex2rsid.csv', squeeze=True, header=None, index_col=0)

In [ ]:
df['rsid'] = ids[df['rsid'].values].values
df['locus_id'] = ids[df['locus_id'].values].values

In [ ]:
df[df['above_threshold']]

In [ ]:
df.to_csv('%s/paintor.tsv'%paintordir, index=False)

In [ ]:
kilog_dir = join(paintordir,'1000g_ld')
mkdir(kilog_dir)

In [ ]:
for chrom, group in df[df['above_threshold']].groupby('CHR'):
    bim1kg = pd.read_csv('/data/1000g/ref/chr%02d.bim'%chrom, sep='\t', header=None)
    snps = np.intersect1d(group['rsid'].values, bim1kg[1].values)
#     print(len(group['rsid'].values),len(snps))
    ancmd = ('plink --bfile /data/1000g/ref/chr%02d --r2 inter-chr --ld-window-r2 0.8 --ld-snps '
          '%s --out %s/chr%02d'%(chrom, ', '.join(snps), kilog_dir, chrom)).split(' ')
    subprocess.check_call(ancmd)

In [ ]:
df

In [ ]:
snp1kg = pd.concat([pd.read_csv(f,
                    delim_whitespace=True) for f in glob('/data/paintor/paintor_test/1000g_ld/chr*.ld')])

In [ ]:
snp1kg.to_csv('/data/tables/suptable5b.tsv')

In [ ]:
veplist = np.unique(np.hstack((df.loc[df['above_threshold'],'rsid'].values,
                               snp1kg['SNP_B'].values)))

veplist = veplist[[elem[:2]=='rs' for elem in veplist]]

np.savetxt('%s/paintor4vep.txt'%paintordir, veplist, fmt='%s')

In [ ]:
df.loc[df['above_threshold'],'rsid'].to_csv('%s/paintor4vep.txt'%paintordir, index=False, header=None)

In [ ]:
df.to_csv('%s/paintor.tsv'%paintordir, index=False)

In [ ]:
rsidcol = df.pop('rsid')
df['rsid'] = ids.set_index('newindex').loc[rsidcol.values,'rsid'].values
df[df['locus_id'].isin(['2:202204741_T_C','6:396321_C_T','6:161013013_T_C','9:22102165_C_T',
                        '10:114754071_T_C'])&df['above_threshold']].reset_index().drop(['index'], axis=1).to_csv(
    'paintor/paintor.tsv', sep='\t', columns=['chr','pos','rsid','locus_id','Posterior_Prob'], index=True)

In [ ]:
a = pd.read_csv('paintor_1000g/paintor.tsv', sep='\t')
a['rsid'].to_csv('paintor/paintor4vep.txt', index=False, header=None)

#### Make COJO on calls data

In [ ]:
pd.read_pickle('GWAS-2/cox/pheno5/ukb_cal_chr22_v2.pkl')[[0.0,1.0,2.0,3.0]].sum(1)

In [ ]:
def save_calls_to_ma(phenoname):
    inp = pd.concat([pd.read_pickle('GWAS-2/cox/%s/ukb_cal_chr%s_v2.pkl'%(phenoname,f)) for f in range(1,23)])
    inp = pd.concat([inp, pd.DataFrame([elem for elem in inp.reset_index()['index'].apply(lambda x: x.split('_'))],
                 columns=['chr','rsid','0','pos','a','b'], index=inp.index)], axis=1)
    Nsamples = inp[[0.0,1.0,2.0]].sum(1).astype(int)
    inp['EAF'] = (inp[1.0]+inp[2.0]*2)/(Nsamples*2.)
    inp = inp.dropna(subset=['beta','sigma','EAF'], how='any')
    inp['PhenoCount'] = Nsamples
    inp['p'] = (inp['beta']/inp['sigma']).apply(pval)

    try:
        mkdir(join('/home//data/GWAS-2/cojo',phenoname))
    except OSError:
        pass

    pbar = ProgressBar()
    for chrom in pbar(inp['chr'].unique()):
        inp[inp['chr']==chrom].to_csv('/home//data/GWAS-2/cojo/%s/ukb_cal_chr%s_v2.ma'%(phenoname,chrom),
                                                       index=False, sep=' ',
            columns=['rsid','a','b','EAF','beta','sigma','p','PhenoCount'],
            header=['SNP','A1','A2','freq','b','se','p','N'])

In [ ]:
for phenoi in range(1,11):
    phenoname = save_calls_to_ma('pheno%s'%phenoi)

In [ ]:
cojo1 = pd.concat([pd.read_csv(f, sep='\t') for f in glob(
    '/home//data/GWAS-2/cojo/andersen_newfam/ukb_cal_chr*.jma.cojo')])
cojo2 = pd.concat([pd.read_csv(f, sep='\t') for f in glob(
    '/home//data/GWAS-2/cojo/andersen_noC44/ukb_cal_chr*.jma.cojo')])

In [ ]:
cojo1.sort_values(['Chr','bp'])

In [ ]:
cojo2.sort_values(['Chr','bp'])

#### Make COJO on imputed data

In [ ]:
endslice = pd.read_pickle('/home//data/GWAS-2/gwas/cardiometabolic.pkl').reset_index()

In [ ]:
endslice['EAF'].describe()

In [ ]:
indexslice = bigdf[bigdf['emaf']>0.001].index
endslice = summary.loc[indexslice,:].reset_index()

In [ ]:
pbar = ProgressBar()
for chrom in pbar(endslice['Chromosome'].unique()):
    endslice[endslice['Chromosome']==chrom].to_csv(
        '/home//data/GWAS-2/cojo/cardiometabolic/chr%02d.ma'%int(chrom), index=False, sep=' ',
        columns=['newindex','eff_allele','ref_allele','EAF','Slope','SESlope','P','PhenoCount'],
        header=['SNP','A1','A2','freq','b','se','p','N'])

In [ ]:
refdir=/mnt/tmp/10KREFS/10kref_imp4cojo_v3
cojodir=/home//data/GWAS-2/cojo/cancer

In [ ]:
parallel cp {} {}_original ::: $refdir/*bim

for f in $refdir/*bim; do
    echo $f;
    awk '{print $1"\t"$1":"$4"_"$5"_"$6"\t"$3"\t"$4"\t"$5"\t"$6}' $f > tmp.txt;
    mv tmp.txt $f; done

In [ ]:
parallel --eta gcta64 --bfile $refdir/{/.} --maf 0.002 --cojo-file {} --cojo-slct --cojo-p 5e-8 --out {.} ::: $cojodir/chr*ma

#### Generate COJO slice

In [ ]:
summary = pd.read_pickle('data/healthspan.pkl')

In [ ]:
cojo = pd.concat([pd.read_csv(f, sep='\t') for f in glob('/home//data/GWAS-2/cojo/andersen_list_v3/chr*.jma.cojo')])

In [ ]:
workslice = summary.loc[cojo['SNP'],['Chromosome','Marker']]

for chrom in workslice['Chromosome'].unique():
        print('plink2 --pfile /mnt/10kref/merged/chr%02d --snps %s --export A --out '
          '/home//data/GWAS-2/cojo/andersen_list_v3/chr%02d' % (int(chrom),
                    ' '.join(workslice.loc[workslice['Chromosome']==chrom,'Marker'].values),int(chrom)))